In [2]:
from transformers import TFCamembertModel,  CamembertConfig, CamembertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

In [1]:
import sys 
sys.path.append("../src/")
from utils.requester import Requester

In [10]:
R = Requester()

l = R.get_aids_mt()

In [12]:
l[0]

{'sourceId': 'at_94683',
 'name': 'Financer les études et tests préalables aux investissements servant à incorporer des matières premières issues du recyclage',
 'perimeter': 'NATIONAL',
 'aidDetails': "<p>\n <strong>\n  Aide - « Études et tests préalables aux investissements pour incorporer des matières premières issues du recyclage », mis en place par l'ADEME\n </strong>\n</p>\n<p>\n Objectif : vous souhaitez étudier ou tester une unité de production intégrant des Matières premières de recyclage (MPR), ou alors adapter une unité existante ? L'ADEME peut vous aider à financer les études et les tests nécessaires à l'expérimentation.\n</p>\n<p>\n Bénéficiaires :\r\nEntreprises de production susceptibles d'utiliser des MPR dans les secteurs de des métaux, du textile, du bâtiment, de l'industrie du papier-carton, du bois...\n <br/>\n <br/>\n Accompagnement : subvention. Taux d'aide maximum : 70 % des dépenses éligibles.\n <br/>\n</p>",
 'eligibility': "<p>\n Critères d'éligibilité\n <br/>

In [5]:
len(l)

349

In [6]:
t = R.get_themes_mt()

In [9]:
l = []

for theme in t:
    tmp = theme["environmentalTopics"]
    for subtheme in tmp:
        l.append(subtheme["id"])
import numpy as np 
print(len(l))
print(np.unique(l).shape)

54
(51,)


In [2]:
def toStr(x):
    if pd.isna(x):
        return ""
    else:
        x = str(x).replace("\n", " ").strip()
        return x

In [3]:
df = pd.read_csv("../data/aids_mt.csv", sep="\t")

In [4]:
df.columns

Index(['name', 'perimeter', 'goal', 'beneficiary', 'aidDetails', 'eligibility',
       'conditions', 'fundingSourceUrl', 'applicationEndDate',
       'applicationUrl', 'slug', 'environmentalTopics', 'funder',
       'fundingTypes', 'regions', 'contactGuidelines',
       'subventionRateUpperBound', 'subventionRateLowerBound', 'loanAmount',
       'applicationStartDate', 'projectExamples', 'directAccess', 'types',
       'id', 'subclasses', 'countSubclasses', 'classes'],
      dtype='object')

In [32]:
df = pd.read_csv("../data/aids_mt.csv", sep="\t")

In [33]:
from bs4 import BeautifulSoup
df["name"] = df["name"].apply(lambda x: toStr(x))
df["aidDetails"] = df["aidDetails"].apply(lambda x: BeautifulSoup(toStr(x)).get_text())
df["eligibility"] = df["eligibility"].apply(lambda x: BeautifulSoup(toStr(x)).get_text())

In [34]:
ok_cols = ["name", "aidDetails", "eligibility", "classes", "subclasses"]

df = df[ok_cols]

In [35]:
df

,name,aidDetails,eligibility,classes,subclasses
0,Financer les études et tests préalables aux in...,Aide - « Études et tests préalables aux in...,Critères d'éligibilité Pertinence du ...,[1],[1]
1,Soutenir financièrement le développement de l'...,Le dispositif de l'ADEME – ORPLAST (Objectif...,"Sont éligibles, les dossiers portant sur u...",[1],[1]
2,Financer les études de préfiguration d'une dém...,Aide - « Étude de préfiguration d'une démarc...,,[1],[3]
3,Réaliser des investissements de décarbonation ...,Plusieurs dispositifs sont ouverts aux entre...,"Pour que la demande d'aide soit éligible, ...",[1],[3]
4,Financer des audits énergétiques en entreprise...,Aide - « Financement des audits énergétiqu...,Critères d'éligibilité : u ne étude de fais...,[1 4],"[6, 7]"
...,...,...,...,...,...
385,Soutenir l'émergence de solutions innovantes p...,La Région soutient l'émergence de solution...,,[3 7],"[47, 5, 4, 10]"
386,Sortir de vacance les logements locatifs privés,Le présent dispositif vise à remettre sur le...,Bénéficiaires éligibles : Proprié...,[7],[24]
387,Soutenir l'économie de proximité en finançant ...,Aide - « Fonds régional des territoires (F...,,[3],[10]
388,Soutenir le développement d'investissements ag...,La Région Occitanie est aux côtés des acteur...,Bénéficiaires Exploitations agricole...,[7],[30]


In [36]:
import sys 
sys.path.append("../")
from src.utils.requester import Requester

R = Requester()

In [37]:
T = R.get_themes_mt()

In [38]:
df[df.classes.str.contains(" 7")]

,name,aidDetails,eligibility,classes,subclasses
30,Investir dans des entreprises à technologies v...,La BPI investit dans des entreprises pour fi...,Vous recherchez des investisseurs en capital...,[3 4 5 6 7 9],"[5, 4, 10, 11, 22, 12, 13, 21, 23, 7, 15, 16, ..."
31,Pour investir au capital de jeunes start-ups -...,"La BPI intervient en capital amorçage, capit...",Vous développez un modèle d'accélérateur pou...,[3 4 5 6 7 9],"[5, 4, 10, 11, 22, 12, 13, 21, 23, 7, 15, 16, ..."
34,Soutenir les technologies et les services dans...,La BPI intervient en capital amorçage et cap...,Vous développez des technologies et des serv...,[3 4 5 7],"[5, 4, 10, 11, 30, 7, 22, 12, 13]"
70,Trouver un plan d'investissement au service de...,____________________________________________...,Vous êtes une collectivité ou un de...,[5 7 8 9],"[21, 23, 22, 12, 13, 25, 19, 29, 30]"
91,Bénéficier d'un prêt vert,Dans le cadre du Plan d'accélération de la t...,Le Prêt Vert s'adresse aux TPE et PME locali...,[ 1 3 5 6 7 8 9 10],"[22, 12, 13, 21, 23, 8, 1, 9, 3, 5, 4, 10, 11,..."
93,Soutenir l'investissement dans les territoires...,La crise sanitaire a profondément affecté no...,Les projets peuvent être de nature individue...,[ 1 3 4 5 7 8 9 10],"[22, 12, 13, 21, 23, 14, 7, 46, 20, 8, 1, 9, 3..."
115,Éduquer la jeunesse au développement durable,Les ressources naturelles de notre Planète n...,Les critères d'éligibilité sont impératifs. ...,[ 7 10],"[41, 40]"
134,Etre aidé dans la mise en oeuvre des villes et...,Les agences d'urbanisme vous aident ...,,[3 4 5 7],"[40, 30, 22, 12, 13, 14, 7, 46, 4, 5]"
139,Obtenir un soutien pour les projets Territoire...,__________________________________________...,Vous êtes Une collectivité locale o...,[3 4 5 6 7 8],"[24, 22, 12, 13, 14, 7, 46, 20, 5, 4, 10, 11, ..."
168,Optimiser la consommation et de la gestion des...,Objectifs : Soutien à l'optimisation de...,,[1 4 6 7 8],"[7, 8, 1, 9, 3, 15, 16, 17, 18, 19]"


In [39]:
new_df = df[df.classes.str.contains(" 1 ")]
new_df.head()

,name,aidDetails,eligibility,classes,subclasses
61,Financer vos projets de transition écologique ...,Financer vos projets de transition écologi...,"L'aide est destinée à toutes les TPE et PME,...",[ 1 4 5 6 9 11],"[23, 7, 22, 15, 17, 9, 8, 3, 37]"
91,Bénéficier d'un prêt vert,Dans le cadre du Plan d'accélération de la t...,Le Prêt Vert s'adresse aux TPE et PME locali...,[ 1 3 5 6 7 8 9 10],"[22, 12, 13, 21, 23, 8, 1, 9, 3, 5, 4, 10, 11,..."
93,Soutenir l'investissement dans les territoires...,La crise sanitaire a profondément affecté no...,Les projets peuvent être de nature individue...,[ 1 3 4 5 7 8 9 10],"[22, 12, 13, 21, 23, 14, 7, 46, 20, 8, 1, 9, 3..."
114,"Trouver des services pour la biodiversité, le ...",__________________________________________...,,[ 1 6 8 10],"[41, 25, 28, 33, 29, 15, 17, 8, 9, 3]"
200,Engager les acteurs du territoire dans la tran...,DESCRIPTION GENERALE ET LOGIQUE D'INTERVEN...,BENEFICIAIRES Opérations de type 9...,[ 1 2 4 5 6 7 8 9 10 11],"[23, 7, 22, 15, 17, 9, 8, 3, 37, 25, 19, 26, 3..."


In [40]:
new_df[new_df.subclasses.str.contains("9")]

,name,aidDetails,eligibility,classes,subclasses
61,Financer vos projets de transition écologique ...,Financer vos projets de transition écologi...,"L'aide est destinée à toutes les TPE et PME,...",[ 1 4 5 6 9 11],"[23, 7, 22, 15, 17, 9, 8, 3, 37]"
91,Bénéficier d'un prêt vert,Dans le cadre du Plan d'accélération de la t...,Le Prêt Vert s'adresse aux TPE et PME locali...,[ 1 3 5 6 7 8 9 10],"[22, 12, 13, 21, 23, 8, 1, 9, 3, 5, 4, 10, 11,..."
93,Soutenir l'investissement dans les territoires...,La crise sanitaire a profondément affecté no...,Les projets peuvent être de nature individue...,[ 1 3 4 5 7 8 9 10],"[22, 12, 13, 21, 23, 14, 7, 46, 20, 8, 1, 9, 3..."
114,"Trouver des services pour la biodiversité, le ...",__________________________________________...,,[ 1 6 8 10],"[41, 25, 28, 33, 29, 15, 17, 8, 9, 3]"
200,Engager les acteurs du territoire dans la tran...,DESCRIPTION GENERALE ET LOGIQUE D'INTERVEN...,BENEFICIAIRES Opérations de type 9...,[ 1 2 4 5 6 7 8 9 10 11],"[23, 7, 22, 15, 17, 9, 8, 3, 37, 25, 19, 26, 3..."
252,"Réussir les transitions vers la sobriété, la r...",InnSocPDL - Innovation sociale et territor...,Il est vivement conseillé de contacter l'ADE...,[ 1 3 4 5 7 8 10],"[22, 12, 13, 14, 7, 46, 20, 8, 1, 9, 3, 5, 4, ..."
253,Développer de nouveaux services et innover - P...,Le programme LEADER est financé par le fonds...,,[ 1 3 4 5 6 7 8 10],"[22, 12, 13, 14, 7, 46, 20, 8, 1, 9, 3, 5, 4, ..."


In [41]:
df[df.classes.str.contains(" 7")]

,name,aidDetails,eligibility,classes,subclasses
30,Investir dans des entreprises à technologies v...,La BPI investit dans des entreprises pour fi...,Vous recherchez des investisseurs en capital...,[3 4 5 6 7 9],"[5, 4, 10, 11, 22, 12, 13, 21, 23, 7, 15, 16, ..."
31,Pour investir au capital de jeunes start-ups -...,"La BPI intervient en capital amorçage, capit...",Vous développez un modèle d'accélérateur pou...,[3 4 5 6 7 9],"[5, 4, 10, 11, 22, 12, 13, 21, 23, 7, 15, 16, ..."
34,Soutenir les technologies et les services dans...,La BPI intervient en capital amorçage et cap...,Vous développez des technologies et des serv...,[3 4 5 7],"[5, 4, 10, 11, 30, 7, 22, 12, 13]"
70,Trouver un plan d'investissement au service de...,____________________________________________...,Vous êtes une collectivité ou un de...,[5 7 8 9],"[21, 23, 22, 12, 13, 25, 19, 29, 30]"
91,Bénéficier d'un prêt vert,Dans le cadre du Plan d'accélération de la t...,Le Prêt Vert s'adresse aux TPE et PME locali...,[ 1 3 5 6 7 8 9 10],"[22, 12, 13, 21, 23, 8, 1, 9, 3, 5, 4, 10, 11,..."
93,Soutenir l'investissement dans les territoires...,La crise sanitaire a profondément affecté no...,Les projets peuvent être de nature individue...,[ 1 3 4 5 7 8 9 10],"[22, 12, 13, 21, 23, 14, 7, 46, 20, 8, 1, 9, 3..."
115,Éduquer la jeunesse au développement durable,Les ressources naturelles de notre Planète n...,Les critères d'éligibilité sont impératifs. ...,[ 7 10],"[41, 40]"
134,Etre aidé dans la mise en oeuvre des villes et...,Les agences d'urbanisme vous aident ...,,[3 4 5 7],"[40, 30, 22, 12, 13, 14, 7, 46, 4, 5]"
139,Obtenir un soutien pour les projets Territoire...,__________________________________________...,Vous êtes Une collectivité locale o...,[3 4 5 6 7 8],"[24, 22, 12, 13, 14, 7, 46, 20, 5, 4, 10, 11, ..."
168,Optimiser la consommation et de la gestion des...,Objectifs : Soutien à l'optimisation de...,,[1 4 6 7 8],"[7, 8, 1, 9, 3, 15, 16, 17, 18, 19]"


In [42]:
import ast 

df = df.reset_index(drop=True)
for i in range(df.shape[0]):
    for k in range(51):
        tmp = df.loc[i,"subclasses"].replace("[", "").replace("]", "").replace(",", "").split(" ")
        tmp = [int(elt)-1 for elt in tmp if elt != ""]
        if k in tmp:
            df.loc[i,k] = 1
        else:
            df.loc[i,k] = 0
            

In [26]:
df = df.drop(["classes", "subclasses"], axis=1)

In [45]:
T[0]

{'id': 1,
 'name': 'Evolution des procédés industriels',
 'environmentalTopics': [{'name': 'Incorporation de produits recyclés',
   'id': 1},
  {'name': 'Réduction du gaspillage', 'id': 9},
  {'name': "Réduction de la consommation d'énergie", 'id': 6},
  {'name': "Réduction de l'empreinte carbone", 'id': 3},
  {'name': 'Réduction des pollutions & dépollution', 'id': 8}],
 'description': 'Exemples de projets :\r\nEfficacité énergétique des procédés industriels, audit énergétique pour l’industrie, réduction des pollutions industrielles, décarbonation des outils industriels.'}

In [48]:
df = df[(df[0]==1)|(df[8]==1)|(df[5]==1)|(df[2]==1)|(df[7]==1)]

In [50]:
DF = df[[0, 2, 5, 7, 8]]

In [55]:
DF = df.drop(["classes", "subclasses"], axis=1)

In [56]:
DF = DF.drop([i for i in range(51) if i not in [0, 2, 5, 7, 8]], axis=1)

In [70]:
max_length = 512
DF["text"] = DF[["name", "aidDetails"]].apply(lambda x: ". ".join(x), axis=1).reset_index(drop=True)

In [74]:
DF["text"] = DF["name"]

In [61]:
DF.columns = ["name", "aidDetails", "eligibility", 0, 1, 2, 3, 4]

In [43]:
df[range(49)]

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df = pd.read_csv("../src/processings_train.csv", sep="\t", encoding="utf-16")

In [14]:
df

,sourceId,id,name,aidDetails,environmentalTopics,text,0,1,2,3,4,5,6,7,8,9,10,11,list
0,at_94683,189,Financer les études et tests préalables aux in...,Aide - « Études et tests préalables aux in...,[{'name': 'Incorporation de produits recyclés'...,Financer les études et tests préalables aux in...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,at_58447,178,Soutenir financièrement le développement de l'...,Le dispositif de l'ADEME – ORPLAST (Objectif...,[{'name': 'Incorporation de produits recyclés'...,Soutenir financièrement le développement de l'...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,at_94687,203,Financer les études de préfiguration d'une dém...,Aide - « Étude de préfiguration d'une démarc...,"[{'name': ""Réduction de l'empreinte carbone"", ...",Financer les études de préfiguration d'une dém...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,at_71467,150,Réaliser des investissements de décarbonation ...,Plusieurs dispositifs sont ouverts aux entre...,"[{'name': ""Réduction de l'empreinte carbone"", ...",Réaliser des investissements de décarbonation ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,at_94689,161,Financer des audits énergétiques en entreprise...,Aide - « Financement des audits énergétiqu...,"[{'name': ""Réduction de la consommation d'éner...",Financer des audits énergétiques en entreprise...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,at_74159,26,Accompagner les établissements publics de coop...,L'Alliance Inter-métropolitaine Loire Bretag...,"[{'name': 'Formation & Accompagnement', 'id': ...",Accompagner les établissements publics de coop...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
345,at_68134,280,Soutenir l'émergence de solutions innovantes p...,La Région soutient l'émergence de solution...,"[{'name': 'Logistique de proximité', 'id': 47}...",Soutenir l'émergence de solutions innovantes p...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
346,at_97332,96,Sortir de vacance les logements locatifs privés,Le présent dispositif vise à remettre sur le...,[{'name': 'Réhabilitation logements & sites in...,Sortir de vacance les logements locatifs privé...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
347,at_66858,64,Soutenir le développement d'investissements ag...,La Région Occitanie est aux côtés des acteur...,[{'name': 'Animation & développement économiqu...,Soutenir le développement d'investissements ag...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [34]:
df["list"] = df[range(51)].apply(lambda x: [e for e in x], axis=1)

In [62]:
DF["list"] = DF[range(5)].apply(lambda x: [e for e in x], axis=1)

In [63]:
DF

,name,aidDetails,eligibility,0,1,2,3,4,list
0,Financer les études et tests préalables aux in...,Aide - « Études et tests préalables aux in...,Critères d'éligibilité Pertinence du ...,1.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]"
1,Soutenir financièrement le développement de l'...,Le dispositif de l'ADEME – ORPLAST (Objectif...,"Sont éligibles, les dossiers portant sur u...",1.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0]"
2,Financer les études de préfiguration d'une dém...,Aide - « Étude de préfiguration d'une démarc...,,0.0,1.0,0.0,0.0,0.0,"[0.0, 1.0, 0.0, 0.0, 0.0]"
3,Réaliser des investissements de décarbonation ...,Plusieurs dispositifs sont ouverts aux entre...,"Pour que la demande d'aide soit éligible, ...",0.0,1.0,0.0,0.0,0.0,"[0.0, 1.0, 0.0, 0.0, 0.0]"
4,Financer des audits énergétiques en entreprise...,Aide - « Financement des audits énergétiqu...,Critères d'éligibilité : u ne étude de fais...,0.0,0.0,1.0,0.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0]"
5,Financer les équipements de lutte contre le ga...,Aide - « Équipements de lutte contre le ga...,Critères d'éligibilité : Pour prétend...,0.0,0.0,0.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 1.0]"
58,Financer des études de faisabilité ou la démon...,Le Fonds d'études et d'aide au secteur privé...,Toutes les entreprises sont éligibles au FAS...,0.0,0.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 1.0, 1.0]"
61,Financer vos projets de transition écologique ...,Financer vos projets de transition écologi...,"L'aide est destinée à toutes les TPE et PME,...",0.0,1.0,0.0,1.0,1.0,"[0.0, 1.0, 0.0, 1.0, 1.0]"
91,Bénéficier d'un prêt vert,Dans le cadre du Plan d'accélération de la t...,Le Prêt Vert s'adresse aux TPE et PME locali...,1.0,1.0,0.0,1.0,1.0,"[1.0, 1.0, 0.0, 1.0, 1.0]"
92,Financer les équipements éligibles aux Certifi...,Le Prêt Eco-Energie (PEE) permet de financer...,"Peuvent bénéficier du dispositif, les TPE et...",1.0,1.0,0.0,1.0,1.0,"[1.0, 1.0, 0.0, 1.0, 1.0]"


In [75]:
DF[["text", "list"]].to_csv("../data/aids_mt_preprocessed_subclasses_C1.csv", sep="\t", index=False)

In [77]:
DF.text

0      Financer les études et tests préalables aux in...
1      Soutenir financièrement le développement de l'...
2      Financer les études de préfiguration d'une dém...
3      Réaliser des investissements de décarbonation ...
4      Financer des audits énergétiques en entreprise...
5      Financer les équipements de lutte contre le ga...
58     Financer des études de faisabilité ou la démon...
61     Financer vos projets de transition écologique ...
91                             Bénéficier d'un prêt vert
92     Financer les équipements éligibles aux Certifi...
93     Soutenir l'investissement dans les territoires...
114    Trouver des services pour la biodiversité, le ...
144    Soutenir financièrement l'intégration de matiè...
145    Investir sur les procédés de production exista...
146    Accompagner et soutenir les entreprises dans l...
147    Améliorer l'efficacité énergétique des navires...
148    Accompagner la dépollution des industries et a...
149    Agir collectivement sur 

In [8]:
from transformers import CamembertForSequenceClassification, CamembertConfig, CamembertTokenizerFast

model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=12)
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = model.to("cuda")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [37]:
max_length = 512
df["text"] = df[["name", "aidDetails"]].apply(lambda x: ". ".join(x), axis=1)

In [10]:
df = df[["text", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

In [11]:
df_train, df_test = train_test_split(df, test_size=0.05, random_state=42)

In [12]:
train_texts = df_train.text.tolist()
train_labels = df_train.drop("text", axis=1).values
test_texts = df_test.text.tolist()
test_labels = df_test.drop("text", axis=1).values


In [13]:
train_labels = train_labels.reshape((-1,12))
test_labels = test_labels.reshape((-1,12))


In [14]:
import torch

In [15]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
valid_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")

for k in train_encodings.keys():
    train_encodings[k] = train_encodings[k].int().clone().detach()
for k in valid_encodings.keys():
    valid_encodings[k] = valid_encodings[k].int().clone().detach()

In [16]:
import torch
from torch.utils.data import Dataset
import numpy as np 

class AidsDataset(Dataset):
    def __init__(self, encodings, labels, max_len=512):
        self.labels = np.array(labels)
        self.encodings = encodings
        self.max_len = max_len
        
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
train_dataset = AidsDataset(train_encodings, train_labels)
valid_dataset = AidsDataset(valid_encodings, test_labels)

In [18]:
from sklearn.metrics import accuracy_score, f1_score
from torch import nn

def compute_metrics(pred):
  labels = pred.label_ids
  preds = np.array(nn.Sigmoid()(torch.Tensor(pred.predictions)).detach().cpu().numpy() >= 0.5, dtype=float)
  print("preds: ", preds)
  print("labels: ", labels)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average="macro")
  return {
      'accuracy': acc,
      'f1 score': f1
  }

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=100,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [20]:
from torch import nn

class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [21]:
trainer = MultilabelTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [22]:
# train the model
trainer.train()

***** Running training *****
  Num examples = 370
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 555
  0%|          | 0/555 [00:00<?, ?it/s]C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
 18%|█▊        | 100/555 [00:31<02:15,  3.36it/s]***** Running Evaluation *****
  Num examples = 20
  Batch size = 10


{'loss': 0.6566, 'learning_rate': 1e-05, 'epoch': 0.54}


C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))

 18%|█▊        | 100/555 [00:32<02:15,  3.36it/s]C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


preds:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
labels:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0.

 36%|███▌      | 200/555 [01:02<01:46,  3.34it/s]***** Running Evaluation *****
  Num examples = 20
  Batch size = 10


...

C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))

 90%|█████████ | 500/555 [03:16<01:04,  1.17s/it]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


preds:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
labels:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0.

Model weights saved in ./results\checkpoint-500\pytorch_model.bin
C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
100%|██████████| 555/555 [04:24<00:00,  1.19s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-500 (score: 0.3893605172634125).
100%|██████████| 555/555 [04:24<00:00,  2.10it/s]

{'train_runtime': 264.6535, 'train_samples_per_second': 4.194, 'train_steps_per_second': 2.097, 'train_loss': 0.44381331710128097, 'epoch': 3.0}


TrainOutput(global_step=555, training_loss=0.44381331710128097, metrics={'train_runtime': 264.6535, 'train_samples_per_second': 4.194, 'train_steps_per_second': 2.097, 'train_loss': 0.44381331710128097, 'epoch': 3.0})

In [23]:
inpt = tokenizer.encode_plus(df.loc[0,"text"], add_special_tokens=True, max_length=512, return_tensors="pt")
for k in inpt.keys():
    inpt[k] = inpt[k].int().clone().to("cuda")
out = model(**inpt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [25]:
out.logits

tensor([[-2.2787, -2.6698, -1.5949, -2.1019, -1.0884, -2.4630, -1.3397, -1.9340,
         -2.2757, -1.4232, -3.0292, -2.9219]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [26]:
nn.Sigmoid()(out.logits)

tensor([[0.0929, 0.0648, 0.1687, 0.1089, 0.2519, 0.0785, 0.2076, 0.1263, 0.0932,
         0.1942, 0.0461, 0.0511]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [27]:
df.loc[0]

text    Financer les études et tests préalables aux in...
0                                                     1.0
1                                                     0.0
2                                                     0.0
3                                                     0.0
4                                                     0.0
5                                                     0.0
6                                                     0.0
7                                                     0.0
8                                                     0.0
9                                                     0.0
10                                                    0.0
11                                                    0.0
Name: 0, dtype: object

In [1]:
df


NameError: name 'df' is not defined